In [ ]:
!pip install tensorflow

In [2]:
pip install flask


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install flask-cors


Note: you may need to restart the kernel to use updated packages.


##### <p style="color:red">Only run this cell to launch the backend Python Server</p>

In [ ]:
from flask import Flask, request, jsonify
import numpy as np
from flask_cors import CORS  

import base64
from io import BytesIO
from PIL import Image
from tensorflow.keras.models import load_model

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Define your class labels
class_labels = ['tumor','no_tumor' ]

def preprocess_base64_image(base64_str):
    # Remove header if present
    if ',' in base64_str:
        base64_str = base64_str.split(',')[1]
    
    image_data = base64.b64decode(base64_str)
    img = Image.open(BytesIO(image_data)).convert('RGB')
    img = img.resize((150, 150))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array
model =load_model('./brain_tumor_model.h5')
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        base64_image = data.get('image')

        if not base64_image:
            return jsonify({'error': 'No image provided'}), 400

        img_array = preprocess_base64_image(base64_image)
        prob = model.predict(img_array)[0][0]
        prediction = class_labels[int(prob > 0.5)]

        return jsonify({
            "predicted_class": prediction,
            "probability_of_being_tumor": float(prob)
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run the app
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 19:49:38] "OPTIONS /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step


INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 19:49:38] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 20:49:46] "OPTIONS /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 20:49:46] "POST /predict HTTP/1.1" 200 -
